# Descrição do problema.
 
* Objetivo principal deste projeto será a criação de um modelo de machine learning para prever se uma pessoa que foi contratada por uma determinada empresa vai ficar mais de um ano trabalhando nela ou não.
 
* Os dados foram disponibilizados pela comunidade Flai. Onde eles são compostos pelas seguintes variáveis:
 
 * **func_sexo:** Sexo da pessoa.
 * **func_idade:** Idade do usuário.
 * **func_racacor:** Raça da pessoa segundo a classificação do IBGE.
 * **func_escolaridade:** Escolaridade das pessoas.
 * **func_uf:** Estado da pessoa
 * **func_deficiencia:** Se a pessoa é deficiente ou não.
 * **empresa_porte:** Número de funcionários da empresa.
 * **empresa_setor:** Setor que a empresa trabalha.
 * **contrato_horastrabalho:** Quantidade de horas por semana de trabalho.
 * **contrato_salario:** Valor do salário da pessoa na empresa.
 * **turnover_apos_1_ano:** Se a pessoa trabalhou lá mais de um ano ou não.
 
 
 
 
 


# Objetivo

* Será visto como as variáveis independentes afetam o desempenho do modelo padrão adotado. Isso será feito com a intenção de selecionar as melhores para treinar o modelo.
 
* Tambeḿ será testado como o modelo vai desempenhar com a criação de novas variáveis.
 
* E por último, será visto se a mudança do limiar vai fazer com que o valor de f2-score melhore.
 
 


# Instalando as bibliotecas

In [ ]:
!pip install category_encoders
!pip install gokinjo

# Bibliotecas

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier #use RandomForestRegressor para problemas de regressão
from sklearn.metrics import f1_score
from category_encoders import CountEncoder
from category_encoders import TargetEncoder
from gokinjo import knn_kfold_extract
from gokinjo import knn_extract
from sklearn.impute import KNNImputer

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Funções

In [ ]:
def resultados(X_train,y_train,linhas_treino,linhas_valid,flag,coluna,suport,C):

  if suport == 'Dataframe':
    X_t, X_v = X_train.loc[linhas_treino], X_train.loc[linhas_valid]
    y_t, y_v = y_train.loc[linhas_treino], y_train.loc[linhas_valid]

  if suport == 'array':
    X_t, X_v = X_train[linhas_treino], X_train[linhas_valid]
    y_t, y_v = y_train[linhas_treino], y_train[linhas_valid]

  if flag == 'CountEncoder':
      cenc = CountEncoder(normalize=True)
      X_t[coluna] = cenc.fit_transform(X_t[coluna],y_t)
      X_v[coluna] = cenc.transform(X_v[coluna])

  if flag == 'TargetEncoder':
      tenc = TargetEncoder()
      X_t[coluna] = tenc.fit_transform(X_t[coluna],y_t)
      X_v[coluna] = tenc.transform(X_v[coluna])

  model = RandomForestClassifier(n_estimators=100,random_state=10)
  model = model.fit(X_t, y_t)
  prf = model.predict_proba(X_v)[:,1]
  prf_ = (prf > C).astype(int)
  acerto_v = f1_score(y_v, prf_)
  print('f1_score',np.format_float_positional(acerto_v,5))
  return acerto_v




def encoder_var(df,flag):
  if flag == 1:
    ind = df.columns[df.dtypes == 'object']
  if flag == 0:
     ind = df.columns[df.dtypes == 'category']

  for i in range(len(ind)):
    enc = LabelEncoder()
    inteiros = enc.fit_transform(df[ind[i]])
    df[ind[i]] = inteiros

  return df


def va_knn(X_train,y_train,linhas_treino,linhas_valid,knn,flag,coluna,C):
  X_t, X_v = X_train.loc[linhas_treino], X_train.loc[linhas_valid]
  y_t, y_v = y_train.loc[linhas_treino], y_train.loc[linhas_valid]

  if flag == 'CountEncoder':
    cenc = CountEncoder(normalize=True)
    X_t[coluna] = cenc.fit_transform(X_t[coluna],y_t)
    X_v[coluna] = cenc.transform(X_v[coluna])

  KNN_feat_train = knn_kfold_extract(X_t.to_numpy(), y_t.to_numpy(), k=knn, normalize='standard',random_state=42)
  KNN_feat_test  = knn_extract(X_t, y_t, X_v, k=knn, normalize='standard')

  for i in range(KNN_feat_train.shape[1]):
    X_t['knn_'+str(i)]= KNN_feat_train[:,i]
    X_v['knn_'+str(i)]= KNN_feat_test[:,i]

  model = RandomForestClassifier(n_estimators=100,random_state=10) 
  model = model.fit(X_t, y_t)
  prf = model.predict_proba(X_v)[:,1]
  prf_ = (prf > C).astype(int)
  acerto_v = f1_score(y_v, prf_)
  del X_t,X_v
  return acerto_v


def tratamento(df, flag):

  df['func_idade_log'] = np.log(df['func_idade'])
  df['contrato_salario_log'] = np.log(df['contrato_salario'])

  if flag == 1:
    df['turnover_apos_1_ano'] = df['turnover_apos_1_ano'].map({'NAO': 0, 'SIM': 1})
  
  df['func_deficiencia'] = df['func_deficiencia'].map({'NAO': 0, 'SIM': 1})
  df['func_sexo'] = df['func_sexo'].map({'FEMININO': 0, 'MASCULINO': 1})

  df = encoder_var(df,1)

  col2 = ['func_idade' ,'contrato_salario_log']
  df = df.drop(columns = col2)

  df['contrato_horastrabalho'].loc[df['contrato_horastrabalho'] == 0] =df['contrato_horastrabalho'].loc[df['contrato_horastrabalho'] > 0].mean()

  df['contrato_horastrabalho']= pd.cut(df["contrato_horastrabalho"], bins= 12)
  
  df['func_racacor'][df['func_racacor'] =='SEM INFO'] = 'INDIGENA'

  print(np.unique(df['contrato_horastrabalho']))

  #df['contrato_salario_sqrt']= df['contrato_salario']**(1/3)

  df = encoder_var(df,0)

  return df

# Importando os dados

In [ ]:
df = pd.read_csv('/content/drive/My Drive/competição flai nova/treino.csv',encoding = "UTF-8", sep = ";",decimal = ',',dayfirst = True)

In [ ]:
#df['contrato_salario']

In [ ]:
df = tratamento(df,1)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


[Interval(0.957, 4.583, closed='right')
 Interval(4.583, 8.167, closed='right')
 Interval(8.167, 11.75, closed='right')
 Interval(11.75, 15.333, closed='right')
 Interval(15.333, 18.917, closed='right')
 Interval(18.917, 22.5, closed='right')
 Interval(22.5, 26.083, closed='right')
 Interval(26.083, 29.667, closed='right')
 Interval(29.667, 33.25, closed='right')
 Interval(33.25, 36.833, closed='right')
 Interval(36.833, 40.417, closed='right')
 Interval(40.417, 44.0, closed='right')]


In [ ]:
df.head()

,func_sexo,func_racacor,func_escolaridade,func_uf,func_deficiencia,empresa_porte,empresa_setor,contrato_horastrabalho,contrato_salario,turnover_apos_1_ano,func_idade_log
0,1,0,2,7,0,2,5,11,1750.65,1,3.178054
1,1,3,2,1,0,1,2,11,1002.30,0,3.178054
2,0,2,2,8,0,4,2,11,1370.14,0,3.218876
3,1,2,2,5,0,4,2,11,1854.68,1,3.988984
4,1,2,2,1,0,4,2,11,1126.23,0,3.688879


# Seleção de variáveis.

In [ ]:
X_train1  = df.drop(columns='turnover_apos_1_ano')
y_train   = df['turnover_apos_1_ano']

In [ ]:
vetor = np.zeros(X_train1.shape[1])

In [ ]:
kf = KFold(n_splits=10, random_state=5, shuffle=True)

### Primeira variável de maior importância
 
 


In [ ]:
for i in range(X_train1.shape[1]):
  acerto = np.zeros(10)
  j = 0
  print(X_train1.columns[i])
  for linhas_treino, linhas_valid in kf.split(X_train1):
    acerto[j] =  resultados(X_train1[X_train1.columns[i]].values.reshape(-1, 1),y_train.values,linhas_treino,linhas_valid,0,0,'array',0.5)
    j =  j+1
  vetor[i] = np.format_float_positional(np.mean(acerto),5)
  
  print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto),5),' erro:',np.format_float_positional(np.std(acerto)/np.sqrt(10),5),' desvio padrão:',np.format_float_positional(np.std(acerto),5))
  print('\n')

func_sexo
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
dataset est_1 f1-score:     0.  erro: 0.  desvio padrão: 0.


func_racacor
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.00364
f1_score 0.00371
f1_score 0.00353
f1_score 0.
f1_score 0.
dataset est_1 f1-score:     0.00109  erro: 0.00053  desvio padrão: 0.00166


func_escolaridade
f1_score 0.
f1_score 0.28748
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
dataset est_1 f1-score:     0.02875  erro: 0.02727  desvio padrão: 0.08625


func_uf
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
dataset est_1 f1-score:     0.  erro: 0.  desvio padrão: 0.


func_deficiencia
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
f1_score 0.
dataset est_1 f1-score:     0.  erro

* Melhor variável contrato_salario
 
### Segunda variável de maior importância.
 
 


In [ ]:
X = X_train1.copy()

In [ ]:
col = ['func_sexo', 'func_racacor', 'func_escolaridade', 'func_uf',
       'func_deficiencia', 'empresa_porte', 'empresa_setor',
        'contrato_horastrabalho', 'func_idade_log']

In [ ]:
for i in range(len(col)):
  acerto = np.zeros(10)
  j = 0
  print(col[i])
  for linhas_treino, linhas_valid in kf.split(X_train1):
    
    acerto[j] = resultados(X[['contrato_salario',col[i]]],y_train,linhas_treino,linhas_valid,0,0,'Dataframe',0.5)
    j =  j+1

  vetor[i] = np.format_float_positional(np.mean(acerto),5)
  
  print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto),5),' erro:',np.format_float_positional(np.std(acerto)/np.sqrt(10),5),' desvio padrão:',np.format_float_positional(np.std(acerto),5))
  print('\n')

func_sexo
f1_score 0.65350
f1_score 0.6305
f1_score 0.65152
f1_score 0.65549
f1_score 0.6525
f1_score 0.67157
f1_score 0.66291
f1_score 0.64977
f1_score 0.65018
f1_score 0.6548
dataset est_1 f1-score:     0.65327  erro: 0.00313  desvio padrão: 0.0099


func_racacor
f1_score 0.63822
f1_score 0.65693
f1_score 0.64451
f1_score 0.65924
f1_score 0.65018
f1_score 0.67403
f1_score 0.63628
f1_score 0.60797
f1_score 0.63645
f1_score 0.62908
dataset est_1 f1-score:     0.64329  erro: 0.00548  desvio padrão: 0.01734


func_escolaridade
f1_score 0.656
f1_score 0.65257
f1_score 0.65363
f1_score 0.65279
f1_score 0.67325
f1_score 0.67288
f1_score 0.65177
f1_score 0.64147
f1_score 0.63873
f1_score 0.61919
dataset est_1 f1-score:     0.65123  erro: 0.00476  desvio padrão: 0.01505


func_uf
f1_score 0.6411
f1_score 0.65610
f1_score 0.66178
f1_score 0.65666
f1_score 0.65150
f1_score 0.65738
f1_score 0.63688
f1_score 0.63004
f1_score 0.64534
f1_score 0.64971
dataset est_1 f1-score:     0.64865  erro: 0.00

* Segunda variável contrato_horastrabalho
 
 
### Terceira variável de maior importância.
 
 


In [ ]:
col = ['func_sexo', 'func_racacor', 'func_escolaridade', 'func_uf',
       'func_deficiencia', 'empresa_porte', 'empresa_setor',
        'func_idade_log']

In [ ]:
for i in range(len(col)):
  acerto = np.zeros(10)
  j = 0
  print(col[i])
  for linhas_treino, linhas_valid in kf.split(X_train1):
    acerto[j] = resultados(X[['contrato_salario','contrato_horastrabalho',col[i]]],y_train,linhas_treino,linhas_valid,0,0,'Dataframe',0.5)
    j =  j+1

  vetor[i] = np.format_float_positional(np.mean(acerto),5)
  print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto),5),' erro:',np.format_float_positional(np.std(acerto)/np.sqrt(10),5),' desvio padrão:',np.format_float_positional(np.std(acerto),5))
  print('\n')

func_sexo
f1_score 0.67147
f1_score 0.68089
f1_score 0.68347
f1_score 0.68093
f1_score 0.66667
f1_score 0.68450
f1_score 0.66667
f1_score 0.68297
f1_score 0.66482
f1_score 0.66601
dataset est_1 f1-score:     0.67484  erro: 0.00251  desvio padrão: 0.00794


func_racacor
f1_score 0.68491
f1_score 0.70265
f1_score 0.66667
f1_score 0.67530
f1_score 0.67097
f1_score 0.67218
f1_score 0.65714
f1_score 0.66789
f1_score 0.66728
f1_score 0.63473
dataset est_1 f1-score:     0.66997  erro: 0.00525  desvio padrão: 0.0166


func_escolaridade
f1_score 0.68097
f1_score 0.67279
f1_score 0.69581
f1_score 0.67989
f1_score 0.67216
f1_score 0.68566
f1_score 0.67959
f1_score 0.68368
f1_score 0.64372
f1_score 0.65126
dataset est_1 f1-score:     0.67455  erro: 0.00475  desvio padrão: 0.01502


func_uf
f1_score 0.66788
f1_score 0.68767
f1_score 0.71103
f1_score 0.69557
f1_score 0.67695
f1_score 0.67099
f1_score 0.65847
f1_score 0.67212
f1_score 0.67166
f1_score 0.68695
dataset est_1 f1-score:     0.67993  erro

* Terceira variável de maior importância empresa_setor
 
### Quarta variável de maior importância.
 
 


In [ ]:
col = ['func_sexo', 'func_racacor', 'func_escolaridade', 'func_uf',
       'func_deficiencia', 'empresa_porte',
        'func_idade_log']

In [ ]:
for i in range(len(col)):
  acerto = np.zeros(10)
  j = 0
  print(col[i])
  for linhas_treino, linhas_valid in kf.split(X_train1):
    acerto[j] = resultados(X[['contrato_salario','contrato_horastrabalho','empresa_setor',col[i]]],y_train,linhas_treino,linhas_valid,0,0,'Dataframe',0.5)
    j =  j+1

  vetor[i] = np.format_float_positional(np.mean(acerto),5)
  
  print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto),5),' erro:',np.format_float_positional(np.std(acerto)/np.sqrt(10),5),' desvio padrão:',np.format_float_positional(np.std(acerto),5))
  print('\n')

func_sexo
f1_score 0.72484
f1_score 0.69421
f1_score 0.70169
f1_score 0.69303
f1_score 0.68421
f1_score 0.70195
f1_score 0.69216
f1_score 0.70641
f1_score 0.67655
f1_score 0.70763
dataset est_1 f1-score:     0.69827  erro: 0.00404  desvio padrão: 0.01279


func_racacor
f1_score 0.7041
f1_score 0.71688
f1_score 0.70285
f1_score 0.69145
f1_score 0.67642
f1_score 0.69452
f1_score 0.69266
f1_score 0.66913
f1_score 0.67552
f1_score 0.68214
dataset est_1 f1-score:     0.69057  erro: 0.00448  desvio padrão: 0.01416


func_escolaridade
f1_score 0.70786
f1_score 0.72080
f1_score 0.71508
f1_score 0.70260
f1_score 0.70642
f1_score 0.70066
f1_score 0.695
f1_score 0.70898
f1_score 0.66914
f1_score 0.6998
dataset est_1 f1-score:     0.70263  erro: 0.00419  desvio padrão: 0.01326


func_uf
f1_score 0.72529
f1_score 0.71131
f1_score 0.71795
f1_score 0.71985
f1_score 0.70652
f1_score 0.68997
f1_score 0.68721
f1_score 0.68066
f1_score 0.69346
f1_score 0.71045
dataset est_1 f1-score:     0.70427  erro: 0

* quarta variável de maior importância func_idade_log
 
### Quinta variável de maior importância.
 
 


In [ ]:
col = ['func_sexo', 'func_racacor', 'func_escolaridade', 'func_uf',
       'func_deficiencia', 'empresa_porte']

In [ ]:
for i in range(len(col)):
  acerto = np.zeros(10)
  j = 0
  print(col[i])
  for linhas_treino, linhas_valid in kf.split(X_train1):
    acerto[j] = resultados(X[['contrato_salario','contrato_horastrabalho','empresa_setor','func_idade_log',col[i]]],y_train,linhas_treino,linhas_valid,0,0,'Dataframe',0.5)
    j =  j+1

  vetor[i] = np.format_float_positional(np.mean(acerto),5)
  print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto),5),' erro:',np.format_float_positional(np.std(acerto)/np.sqrt(10),5),' desvio padrão:',np.format_float_positional(np.std(acerto),5))
  print('\n')

func_sexo
f1_score 0.7305
f1_score 0.70631
f1_score 0.75203
f1_score 0.74643
f1_score 0.71008
f1_score 0.72939
f1_score 0.7185
f1_score 0.75000
f1_score 0.70401
f1_score 0.71486
dataset est_1 f1-score:     0.72621  erro: 0.00548  desvio padrão: 0.01734


func_racacor
f1_score 0.73852
f1_score 0.72352
f1_score 0.72379
f1_score 0.72247
f1_score 0.713
f1_score 0.72954
f1_score 0.73091
f1_score 0.72438
f1_score 0.72119
f1_score 0.70788
dataset est_1 f1-score:     0.72352  erro: 0.00260  desvio padrão: 0.00824


func_escolaridade
f1_score 0.72566
f1_score 0.72414
f1_score 0.75495
f1_score 0.75918
f1_score 0.73197
f1_score 0.74592
f1_score 0.72858
f1_score 0.73592
f1_score 0.70120
f1_score 0.71330
dataset est_1 f1-score:     0.73208  erro: 0.00537  desvio padrão: 0.01699


func_uf
f1_score 0.75196
f1_score 0.72352
f1_score 0.74823
f1_score 0.74843
f1_score 0.72069
f1_score 0.72598
f1_score 0.73703
f1_score 0.73749
f1_score 0.72015
f1_score 0.71528
dataset est_1 f1-score:     0.73288  erro: 0

* Quinta variável de maior importância empresa_porte
 
### Sexta variável de maior importância
 
 


In [ ]:
col = ['func_sexo', 'func_racacor', 'func_escolaridade', 'func_uf',
       'func_deficiencia']

In [ ]:
for i in range(len(col)):
  acerto = np.zeros(10)
  j = 0
  print(col[i])
  for linhas_treino, linhas_valid in kf.split(X_train1):
    acerto[j] = resultados(X[['contrato_salario','contrato_horastrabalho','empresa_setor','func_idade_log','empresa_porte',col[i]]],y_train,linhas_treino,linhas_valid,0,0,'Dataframe',0.5)
    j =  j+1

  vetor[i] = np.format_float_positional(np.mean(acerto),5)
  print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto),5),' erro:',np.format_float_positional(np.std(acerto)/np.sqrt(10),5),' desvio padrão:',np.format_float_positional(np.std(acerto),5))
  print('\n')

func_sexo
f1_score 0.76923
f1_score 0.74402
f1_score 0.77748
f1_score 0.76377
f1_score 0.73145
f1_score 0.75067
f1_score 0.74889
f1_score 0.76329
f1_score 0.71402
f1_score 0.75315
dataset est_1 f1-score:     0.7516  erro: 0.00561  desvio padrão: 0.01775


func_racacor
f1_score 0.74341
f1_score 0.76074
f1_score 0.77526
f1_score 0.74468
f1_score 0.74301
f1_score 0.74912
f1_score 0.7509
f1_score 0.76273
f1_score 0.70837
f1_score 0.74662
dataset est_1 f1-score:     0.74848  erro: 0.00525  desvio padrão: 0.01661


func_escolaridade
f1_score 0.75
f1_score 0.76274
f1_score 0.78044
f1_score 0.77171
f1_score 0.73852
f1_score 0.76054
f1_score 0.75224
f1_score 0.75713
f1_score 0.72978
f1_score 0.75512
dataset est_1 f1-score:     0.75582  erro: 0.00442  desvio padrão: 0.01398


func_uf
f1_score 0.77702
f1_score 0.74106
f1_score 0.77465
f1_score 0.75782
f1_score 0.74324
f1_score 0.76522
f1_score 0.75177
f1_score 0.76109
f1_score 0.72793
f1_score 0.75170
dataset est_1 f1-score:     0.75515  erro: 0.

* Sexta variável de maior importância é a func_escolaridade
 
### Sétima variável de maior importância.
 
 


In [ ]:
col = ['func_sexo', 'func_racacor', 'func_uf',
       'func_deficiencia']

In [ ]:
for i in range(len(col)):
  acerto = np.zeros(10)
  j = 0
  print(col[i])
  for linhas_treino, linhas_valid in kf.split(X_train1):
    acerto[j] = resultados(X[['contrato_salario','contrato_horastrabalho','empresa_setor','func_idade_log','empresa_porte','func_escolaridade',col[i]]],y_train,linhas_treino,linhas_valid,0,0,'Dataframe',0.5)
    j =  j+1

  vetor[i] = np.format_float_positional(np.mean(acerto),5)
  print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto),5),' erro:',np.format_float_positional(np.std(acerto)/np.sqrt(10),5),' desvio padrão:',np.format_float_positional(np.std(acerto),5))
  print('\n')

func_sexo
f1_score 0.76569
f1_score 0.76148
f1_score 0.79332
f1_score 0.76199
f1_score 0.7478
f1_score 0.75806
f1_score 0.73466
f1_score 0.76936
f1_score 0.72777
f1_score 0.76089
dataset est_1 f1-score:     0.75810  erro: 0.00551  desvio padrão: 0.01742


func_racacor
f1_score 0.76843
f1_score 0.78783
f1_score 0.7767
f1_score 0.78122
f1_score 0.73917
f1_score 0.76476
f1_score 0.75812
f1_score 0.7642
f1_score 0.72694
f1_score 0.74540
dataset est_1 f1-score:     0.76128  erro: 0.00578  desvio padrão: 0.01828


func_uf
f1_score 0.77586
f1_score 0.76815
f1_score 0.77963
f1_score 0.7762
f1_score 0.74632
f1_score 0.76626
f1_score 0.74910
f1_score 0.77015
f1_score 0.74
f1_score 0.75749
dataset est_1 f1-score:     0.76292  erro: 0.00417  desvio padrão: 0.01317


func_deficiencia
f1_score 0.76007
f1_score 0.75512
f1_score 0.77768
f1_score 0.76302
f1_score 0.73573
f1_score 0.75022
f1_score 0.73893
f1_score 0.75556
f1_score 0.71956
f1_score 0.75734
dataset est_1 f1-score:     0.75132  erro: 0.004

* Sétima variável de maior importância func_uf
 
### Oitava variável de maior importância
 
 
 
 


In [ ]:
col = ['func_racacor', 'func_sexo',
       'func_deficiencia']


In [ ]:
for i in range(len(col)):
  acerto = np.zeros(10)
  j = 0
  print(col[i])
  for linhas_treino, linhas_valid in kf.split(X_train1):
    acerto[j] = resultados(X[['contrato_salario','contrato_horastrabalho','empresa_setor','func_idade_log','empresa_porte','func_escolaridade','func_uf',col[i]]],y_train,linhas_treino,linhas_valid,0,0,'Dataframe',0.5)
    j =  j+1

  vetor[i] = np.format_float_positional(np.mean(acerto),5)
  
  print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto),5),' erro:',np.format_float_positional(np.std(acerto)/np.sqrt(10),5),' desvio padrão:',np.format_float_positional(np.std(acerto),5))
  print('\n')

func_racacor
f1_score 0.77202
f1_score 0.77153
f1_score 0.79268
f1_score 0.77641
f1_score 0.75472
f1_score 0.77301
f1_score 0.76318
f1_score 0.76632
f1_score 0.72628
f1_score 0.73973
dataset est_1 f1-score:     0.76359  erro: 0.00573  desvio padrão: 0.01812


func_sexo
f1_score 0.77576
f1_score 0.75552
f1_score 0.80035
f1_score 0.77649
f1_score 0.75582
f1_score 0.77442
f1_score 0.75782
f1_score 0.77653
f1_score 0.74385
f1_score 0.74901
dataset est_1 f1-score:     0.76656  erro: 0.00512  desvio padrão: 0.0162


func_deficiencia
f1_score 0.76857
f1_score 0.75989
f1_score 0.77993
f1_score 0.76565
f1_score 0.75533
f1_score 0.77003
f1_score 0.75269
f1_score 0.76830
f1_score 0.73171
f1_score 0.73973
dataset est_1 f1-score:     0.75918  erro: 0.00442  desvio padrão: 0.01397




* Oitava variável de maior importancia func_sexo
 
 
### Nona variável de maior importância.
 
 


In [ ]:
col = ['func_racacor',
       'func_deficiencia']

In [ ]:
acerto = np.zeros((2,10))
for i in range(len(col)):
  j = 0
  print(col[i])
  for linhas_treino, linhas_valid in kf.split(X_train1):
    acerto[i,j] = resultados(X[['contrato_salario','contrato_horastrabalho','empresa_setor','func_idade_log','empresa_porte','func_escolaridade','func_sexo','func_uf',col[i]]],y_train,linhas_treino,linhas_valid,0,0,'Dataframe',0.5)
    j =  j+1

  vetor[i] = np.format_float_positional(np.mean(acerto[i]),5)
  
  print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto[i]),5),
        ' erro:',np.format_float_positional(np.std(acerto[i])/np.sqrt(10),5),
        ' desvio padrão:',np.format_float_positional(np.std(acerto[i]),5))
  print('\n')

func_racacor
f1_score 0.79265
f1_score 0.77459
f1_score 0.8049
f1_score 0.77993
f1_score 0.76738
f1_score 0.78276
f1_score 0.76258
f1_score 0.78632
f1_score 0.74094
f1_score 0.75796
dataset est_1 f1-score:     0.77500  erro: 0.00555  desvio padrão: 0.01755


func_deficiencia
f1_score 0.77998
f1_score 0.77058
f1_score 0.80070
f1_score 0.77059
f1_score 0.75928
f1_score 0.77857
f1_score 0.75918
f1_score 0.78216
f1_score 0.73933
f1_score 0.74052
dataset est_1 f1-score:     0.76809  erro: 0.00572  desvio padrão: 0.01808




* Quando é removida a variável deficiência o desempenho parece que existe uma pequena melhora, mas pode ser que ela esteja acontecendo por causa da flutuação estatística. Logo para melhorar a estabilidade do resultado será aumentado o número de árvores para ver como o modelo está agindo.
 
 
 
 


### Todas as variáveis

In [ ]:
acerto1 = np.zeros(10)
j =0
for linhas_treino, linhas_valid in kf.split(X_train1):
  acerto1[j] = resultados(X,y_train,linhas_treino,linhas_valid,0,0,'Dataframe',0.5)
  j =  j+1  
print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto1),5),' erro:',np.format_float_positional(np.std(acerto1)/np.sqrt(10),5),' desvio padrão:',np.format_float_positional(np.std(acerto1),5))

### Teste novo var

In [ ]:
acerto1n = np.zeros(10)
j =0
for linhas_treino, linhas_valid in kf.split(X_train1):

  X_t, X_v = X_train1.loc[linhas_treino], X_train1.loc[linhas_valid]
  y_t, y_v = y_train.loc[linhas_treino], y_train.loc[linhas_valid]


  model = RandomForestClassifier(n_estimators=1000,random_state=10)
  model = model.fit(X_t, y_t)
  prf = model.predict_proba(X_v)[:,1]
  prf_ = (prf > 0.5).astype(int)
  acerto1n[j] = f1_score(y_v, prf_)
  print('f1_score',np.format_float_positional(acerto1n[j],5))
  j =  j+1  

print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto1n),5),' erro:',np.format_float_positional(np.std(acerto1n)/np.sqrt(10),5),' desvio padrão:',np.format_float_positional(np.std(acerto1n),5))

f1_score 0.78756
f1_score 0.78147
f1_score 0.80173
f1_score 0.78952
f1_score 0.76264
f1_score 0.78578
f1_score 0.77261
f1_score 0.77863
f1_score 0.74684
f1_score 0.74855
dataset est_1 f1-score:     0.77553  erro: 0.00539  desvio padrão: 0.01704


In [ ]:
X = X_train1.drop(columns = 'func_deficiencia')

In [ ]:
acerto1n = np.zeros(10)
j =0
for linhas_treino, linhas_valid in kf.split(X_train1):

  X_t, X_v = X.loc[linhas_treino], X.loc[linhas_valid]
  y_t, y_v = y_train.loc[linhas_treino], y_train.loc[linhas_valid]


  model = RandomForestClassifier(n_estimators=1000,random_state=10)
  model = model.fit(X_t, y_t)
  prf = model.predict_proba(X_v)[:,1]
  prf_ = (prf > 0.5).astype(int)
  acerto1n[j] = f1_score(y_v, prf_)
  print('f1_score',np.format_float_positional(acerto1n[j],5))
  j =  j+1  

print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto1n),5),' erro:',np.format_float_positional(np.std(acerto1n)/np.sqrt(10),5),' desvio padrão:',np.format_float_positional(np.std(acerto1n),5))

f1_score 0.79414
f1_score 0.78019
f1_score 0.80312
f1_score 0.78841
f1_score 0.76752
f1_score 0.79094
f1_score 0.77261
f1_score 0.77937
f1_score 0.74842
f1_score 0.74203
dataset est_1 f1-score:     0.77668  erro: 0.00589  desvio padrão: 0.01861


* Os resultados mostraram que o descarte da variável deficiente causou uma melhora no desempenho até quando aumentou o número de árvores. Só que de fato esse aumento não foi significativo. Por esse motivo não dá para assumir o descarte dessa variável. 
 


# Criação de variáveis

### Knn

In [ ]:
df = df.drop(columns = 'func_deficiencia')

In [ ]:
X_train1  = df.drop(columns='turnover_apos_1_ano')
y_train   = df['turnover_apos_1_ano']

In [ ]:
#flag,coluna,C

In [ ]:
acerto_k1 = np.zeros(10)
j = 0
for linhas_treino, linhas_valid in kf.split(X_train1):
    acerto_k1[j] = va_knn(X_train1,y_train,linhas_treino,linhas_valid,1,1,0,0.5)
    j =  j+1  
print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto_k1),5),
      ' erro:',np.format_float_positional(np.std(acerto_k1)/np.sqrt(10),5),
      ' desvio padrão:',np.format_float_positional(np.std(acerto_k1),5))

dataset est_1 f1-score:     0.77479  erro: 0.00665  desvio padrão: 0.02102


* O knn não ta adiando nd, não vale apena apostar mais nele

# Melhor valor de corte

In [ ]:
for i in np.arange(0.1,0.91,0.1):
  acerto_c = np.zeros(10)
  j = 0
  print('corte:',i)
  for linhas_treino, linhas_valid in kf.split(X_train1):
      acerto_c[j] = resultados(X,y_train,linhas_treino,linhas_valid,0,0,'Dataframe',i)
      j =  j+1  

  print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto_c),5),
        ' erro:',np.format_float_positional(np.std(acerto_c)/np.sqrt(10),5),
        ' desvio padrão:',np.format_float_positional(np.std(acerto_c),5))
  
  print('\n')

corte: 0.1
f1_score 0.77599
f1_score 0.76596
f1_score 0.77
f1_score 0.76856
f1_score 0.76705
f1_score 0.77266
f1_score 0.76561
f1_score 0.77083
f1_score 0.72570
f1_score 0.73201
dataset est_1 f1-score:     0.76144  erro: 0.00526  desvio padrão: 0.01662


corte: 0.2
f1_score 0.79478
f1_score 0.78556
f1_score 0.79198
f1_score 0.79031
f1_score 0.78074
f1_score 0.79339
f1_score 0.78123
f1_score 0.78902
f1_score 0.75435
f1_score 0.7584
dataset est_1 f1-score:     0.78197  erro: 0.0043  desvio padrão: 0.01359


corte: 0.30000000000000004
f1_score 0.80514
f1_score 0.79847
f1_score 0.79724
f1_score 0.79128
f1_score 0.78506
f1_score 0.80249
f1_score 0.78840
f1_score 0.8
f1_score 0.75829
f1_score 0.77104
dataset est_1 f1-score:     0.78974  erro: 0.00447  desvio padrão: 0.01415


corte: 0.4
f1_score 0.80065
f1_score 0.80096
f1_score 0.81331
f1_score 0.80228
f1_score 0.7801
f1_score 0.79610
f1_score 0.78332
f1_score 0.79937
f1_score 0.75626
f1_score 0.76664
dataset est_1 f1-score:     0.7899  err

In [ ]:
for i in np.arange(0.2,0.61,0.01):
  acerto_c = np.zeros(10)
  j = 0
  print('corte:',i)
  for linhas_treino, linhas_valid in kf.split(X_train1):
      acerto_c[j] = resultados(X,y_train,linhas_treino,linhas_valid,0,0,'Dataframe',i)
      j =  j+1  

  print('corte:',i)
  print('dataset est_1 f1-score:    ',np.format_float_positional(np.mean(acerto_c),5),
        ' erro:',np.format_float_positional(np.std(acerto_c)/np.sqrt(10),5),
        ' desvio padrão:',np.format_float_positional(np.std(acerto_c),5))
  print('\n')

corte: 0.2
f1_score 0.79478
f1_score 0.78556
f1_score 0.79198
f1_score 0.79031
f1_score 0.78074
f1_score 0.79339
f1_score 0.78123
f1_score 0.78902
f1_score 0.75435
f1_score 0.7584
corte: 0.2
dataset est_1 f1-score:     0.78197  erro: 0.0043  desvio padrão: 0.01359


corte: 0.21000000000000002
f1_score 0.79446
f1_score 0.78614
f1_score 0.79254
f1_score 0.78967
f1_score 0.78067
f1_score 0.79248
f1_score 0.78149
f1_score 0.79100
f1_score 0.75455
f1_score 0.75823
corte: 0.21000000000000002
dataset est_1 f1-score:     0.78212  erro: 0.00431  desvio padrão: 0.01361


corte: 0.22000000000000003
f1_score 0.79649
f1_score 0.78698
f1_score 0.79401
f1_score 0.78875
f1_score 0.78151
f1_score 0.79427
f1_score 0.78234
f1_score 0.79330
f1_score 0.75475
f1_score 0.75907
corte: 0.22000000000000003
dataset est_1 f1-score:     0.78315  erro: 0.00443  desvio padrão: 0.014


corte: 0.23000000000000004
f1_score 0.79941
f1_score 0.78810
f1_score 0.79458
f1_score 0.78903
f1_score 0.78209
f1_score 0.79423
f1_s

# Conclusão.
 
* Pelas análises que foram feitas, foram possíveis tirar as seguintes conclusões:
 * Foi visto que a variável deficiênte não causa um impacto significativo no desempenho do modelo padrão. Isso faz com que ela seja candidata para descarte.
 * A mudança do valor de Limiar do modelo realmente causa uma melhora no desempenho. 